In [4]:
import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_API_KEY"),  
    api_version="2025-03-01-preview",
    # azure_endpoint = "https://aisg-sj.openai.azure.com/" # o4-mini
    azure_endpoint = "https://decla-mbncunfi-australiaeast.cognitiveservices.azure.com/" # o3-mini
    )

# Upload a file with a purpose of "batch"
file = client.files.create(
  file=open("example_batch_input.jsonl", "rb"), 
  purpose="batch",
  extra_body={"expires_after":{"seconds": 1209600, "anchor": "created_at"}} # Optional you can set to a number between 1209600-2592000. This is equivalent to 14-30 days
)


print(file.model_dump_json(indent=2))

{
  "id": "file-5a7b2c7922194f5083f7bae7e730f14b",
  "bytes": 1625,
  "created_at": 1749630487,
  "filename": "example_batch_input.jsonl",
  "object": "file",
  "purpose": "batch",
  "status": "processed",
  "expires_at": 1750840087,
  "status_details": null
}


In [5]:
import datetime

print(f"File expiration: {datetime.datetime.fromtimestamp(file.expires_at) if file.expires_at is not None else 'Not set'}")

file_id = file.id

File expiration: 2025-06-25 16:28:07


In [6]:
# Submit a batch job with the file
batch_response = client.batches.create(
    input_file_id=file_id,
    endpoint="/chat/completions",
    completion_window="24h",
    extra_body={"output_expires_after":{"seconds": 1209600, "anchor": "created_at"}} # Optional you can set to a number between 1209600-2592000. This is equivalent to 14-30 days
)


# Save batch ID for later use
batch_id = batch_response.id

print(batch_response.model_dump_json(indent=2))

{
  "id": "batch_d4b38a98-4b8e-4ad0-b667-514f90a7dff4",
  "completion_window": "24h",
  "created_at": 1749630496,
  "endpoint": "/chat/completions",
  "input_file_id": "file-5a7b2c7922194f5083f7bae7e730f14b",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": "",
  "errors": null,
  "expired_at": null,
  "expires_at": 1749716893,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": null,
  "output_file_id": "",
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}


In [7]:
import time
import datetime 

status = "validating"
while status not in ("completed", "failed", "canceled"):
    time.sleep(60)
    batch_response = client.batches.retrieve(batch_id)
    status = batch_response.status
    print(f"{datetime.datetime.now()} Batch Id: {batch_id},  Status: {status}")

if batch_response.status == "failed":
    for error in batch_response.errors.data:  
        print(f"Error code {error.code} Message {error.message}")

2025-06-11 16:29:16.871933 Batch Id: batch_d4b38a98-4b8e-4ad0-b667-514f90a7dff4,  Status: validating
2025-06-11 16:30:17.298923 Batch Id: batch_d4b38a98-4b8e-4ad0-b667-514f90a7dff4,  Status: validating
2025-06-11 16:31:17.726099 Batch Id: batch_d4b38a98-4b8e-4ad0-b667-514f90a7dff4,  Status: in_progress
2025-06-11 16:32:18.166221 Batch Id: batch_d4b38a98-4b8e-4ad0-b667-514f90a7dff4,  Status: in_progress
2025-06-11 16:33:18.590748 Batch Id: batch_d4b38a98-4b8e-4ad0-b667-514f90a7dff4,  Status: finalizing
2025-06-11 16:34:19.021173 Batch Id: batch_d4b38a98-4b8e-4ad0-b667-514f90a7dff4,  Status: completed


In [8]:
import json

output_file_id = batch_response.output_file_id

if not output_file_id:
    output_file_id = batch_response.error_file_id

if output_file_id:
    file_response = client.files.content(output_file_id)
    raw_responses = file_response.text.strip().split('\n')  

    for raw_response in raw_responses:  
        json_response = json.loads(raw_response)  
        formatted_json = json.dumps(json_response, indent=2)  
        print(formatted_json)

{
  "custom_id": "request-1",
  "response": {
    "request_id": "b2488486-f5b0-48c2-a231-89414b3574c6",
    "status_code": 400
  },
  "error": {
    "code": null,
    "message": {
      "error": {
        "message": "Invalid content type. image_url is only supported by certain models.",
        "type": "invalid_request_error",
        "param": "messages.[1].content.[1].type",
        "code": null
      }
    }
  }
}


In [9]:
all_jobs = []
# Automatically fetches more pages as needed.
for job in client.batches.list(
    limit=20,
):
    # Do something with job here
    all_jobs.append(job)
print(all_jobs)

[Batch(id='batch_d4b38a98-4b8e-4ad0-b667-514f90a7dff4', completion_window='24h', created_at=1749630496, endpoint='/chat/completions', input_file_id='file-5a7b2c7922194f5083f7bae7e730f14b', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1749630852, error_file_id='file-f91d073c-a5cf-471f-8385-8ab04b14dc38', errors=None, expired_at=None, expires_at=1749716893, failed_at=None, finalizing_at=1749630768, in_progress_at=1749630655, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=1, total=1))]


In [ ]:
import time
from openai import BadRequestError

max_retries = 10
retries = 0
initial_delay = 5
delay = initial_delay

while True:
    try:
        batch_response = client.batches.create(
            input_file_id=file_id,
            endpoint="/chat/completions",
            completion_window="24h",
        )
        
        # Save batch ID for later use
        batch_id = batch_response.id
        
        print(f"✅ Batch created successfully after {retries} retries")
        print(batch_response.model_dump_json(indent=2))
        break  
        
    except BadRequestError as e:
        error_message = str(e)
        
        # Check if it's a token limit error
        if 'token_limit_exceeded' in error_message:
            retries += 1
            if retries >= max_retries:
                print(f"❌ Maximum retries ({max_retries}) reached. Giving up.")
                raise
            
            print(f"⏳ Token limit exceeded. Waiting {delay} seconds before retry {retries}/{max_retries}...")
            time.sleep(delay)
            
            # Exponential backoff - increase delay for next attempt
            delay *= 2
        else:
            # If it's a different error, raise it immediately
            print(f"❌ Encountered non-token limit error: {error_message}")
            raise